### GWAS and Meta-analysis pipeline
Running cohorts: ALL Cohorts
#### This ipynb notebook is not the latest ver! check .py script.

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange,tqdm
import argparse

In [16]:
Leeds = pd.read_csv("/exports/reum/CKe/Pipelinerun_withPC/Leeds/mergedChr/Leeds_noNA_maf_meta.txt",sep='\t')
Leeds

,CHR,SNP,BP,A1,A2,TEST,NMISS,OR,SE,BETA,L95,U95,STAT,P
0,1,1:729679:C,729679,C,G,ADD,548,1.3070,0.1894,0.267734,0.9015,1.894,1.41300,0.1578
1,1,1:752566:G,752566,G,A,ADD,563,1.3100,0.1794,0.270027,0.9219,1.863,1.50700,0.1318
2,1,1:752721:A,752721,A,G,ADD,563,1.3100,0.1794,0.270027,0.9219,1.863,1.50700,0.1318
3,1,1:753405:C,753405,C,A,ADD,533,1.2800,0.1946,0.246860,0.8737,1.874,1.26600,0.2054
4,1,1:753541:A,753541,A,A,ADD,533,1.2800,0.1946,0.246860,0.8737,1.874,1.26600,0.2054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6706682,22,22:51165721:C,51165721,C,NaN,ADD,563,2.1110,0.7266,0.747162,0.5082,8.771,1.02900,0.3037
6706683,22,22:51169504:A,51169504,A,NaN,ADD,563,2.1110,0.7266,0.747162,0.5082,8.771,1.02900,0.3037
6706684,22,22:51172439:A,51172439,A,NaN,ADD,562,1.4300,0.7810,0.357674,0.3095,6.611,0.45830,0.6467
6706685,22,22:51181759:A,51181759,A,NaN,ADD,563,1.1340,0.1931,0.125751,0.7768,1.656,0.65220,0.5143


In [21]:
Leeds_beforeMeta = pd.read_csv("/exports/reum/CKe/Pipelinerun_withPC/RunMeta/noMAFfilter/Leeds_noNA_noMAFfilter_meta.txt",sep='\t')
Leeds_beforeMeta

,CHR,SNP,BP,A1,A2,TEST,NMISS,OR,SE,BETA,L95,U95,STAT,P
0,1,1:729679:C,729679,C,G,ADD,548,1.3070,0.1894,0.267734,0.9015,1.894,1.41300,0.1578
1,1,1:752566:G,752566,G,A,ADD,563,1.3100,0.1794,0.270027,0.9219,1.863,1.50700,0.1318
2,1,1:752721:A,752721,A,G,ADD,563,1.3100,0.1794,0.270027,0.9219,1.863,1.50700,0.1318
3,1,1:753405:C,753405,C,A,ADD,533,1.2800,0.1946,0.246860,0.8737,1.874,1.26600,0.2054
4,1,1:753541:A,753541,A,A,ADD,533,1.2800,0.1946,0.246860,0.8737,1.874,1.26600,0.2054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6706682,22,22:51165721:C,51165721,C,NaN,ADD,563,2.1110,0.7266,0.747162,0.5082,8.771,1.02900,0.3037
6706683,22,22:51169504:A,51169504,A,NaN,ADD,563,2.1110,0.7266,0.747162,0.5082,8.771,1.02900,0.3037
6706684,22,22:51172439:A,51172439,A,NaN,ADD,562,1.4300,0.7810,0.357674,0.3095,6.611,0.45830,0.6467
6706685,22,22:51181759:A,51181759,A,NaN,ADD,563,1.1340,0.1931,0.125751,0.7768,1.656,0.65220,0.5143


In [25]:
def updateLeeds(dict_a1,cwd=os.getcwd()): #generate text table for --reference-allele input
#     loc_list = getloclist(os.getcwd(),".bim",("_A1corr.bim","_noDup.bim")) #change second param to get only raw files, len=13 (no leeds)
    loc_list = ["/exports/reum/CKe/Pipelinerun_withPC/RunMeta/noMAFfilter/Leeds_noNA_noMAFfilter_meta.txt"]
    for i in range(len(loc_list)):
        cohort_name = 'Leeds'
        print("reading cohort ",cohort_name)
        bim_cohort = pd.read_csv(loc_list[i],sep='\t')
#         bim_cohort.rename(columns={0:"CHR",1:"SNP",3:"BP",4:"A1",5:"A2"},inplace=True)
        bim_cohort['CHRBP'] = bim_cohort['SNP'].str.rsplit(":",expand=True,n=1)[0]
#         update_table = bim_cohort.loc[:,['CHRBP']]   
        if cohort_name != "Tacera": # convert snp id format to chr:bp
            bim_cohort['SNP']=bim_cohort['SNP'].str.split(":",expand=True)[0]+":"+bim_cohort['SNP'].str.split(":",expand=True)[1]
        #bim_cohort.drop_duplicates('SNP', keep = False, inplace=True) #move to further step
        bim_cohort['A1'] = bim_cohort['CHRBP'].map(dict_a1)
        new_loc = loc_list[i].split(".")[0]+"_AllelesCorr.txt"
        bim_cohort.to_csv(new_loc,sep='\t',index=False)
        print("work done on cohort ",cohort_name," count: ",bim_cohort.shape[0])

In [24]:
dict_final_a1

{'1:729679': 'C',
 '1:752566': 'G',
 '1:752721': 'A',
 '1:753405': 'C',
 '1:753541': 'A',
 '1:754182': 'A',
 '1:754192': 'A',
 '1:754334': 'T',
 '1:754503': 'G',
 '1:754964': 'C',
 '1:755775': 'A',
 '1:755890': 'A',
 '1:756604': 'A',
 '1:757640': 'G',
 '1:757734': 'C',
 '1:757936': 'C',
 '1:758144': 'A',
 '1:758626': 'C',
 '1:760912': 'C',
 '1:761147': 'T',
 '1:761732': 'C',
 '1:766007': 'C',
 '1:769223': 'G',
 '1:771823': 'T',
 '1:771967': 'A',
 '1:772755': 'A',
 '1:777122': 'A',
 '1:778745': 'G',
 '1:779322': 'G',
 '1:780785': 'T',
 '1:782981': 'T',
 '1:785050': 'G',
 '1:785989': 'T',
 '1:787606': 'T',
 '1:787844': 'C',
 '1:791191': 'A',
 '1:797440': 'C',
 '1:798400': 'G',
 '1:798959': 'A',
 '1:801467': 'C',
 '1:804759': 'T',
 '1:808631': 'G',
 '1:808928': 'C',
 '1:845635': 'T',
 '1:845938': 'A',
 '1:846078': 'T',
 '1:846398': 'A',
 '1:846808': 'T',
 '1:846864': 'C',
 '1:847228': 'T',
 '1:847491': 'A',
 '1:848090': 'A',
 '1:848445': 'A',
 '1:848456': 'G',
 '1:848738': 'T',
 '1:850062

In [27]:
updateLeeds(dict_final_a1,cwd=os.getcwd())

reading cohort  Leeds
work done on cohort  Leeds  count:  6706687


In [10]:
loc_list = getloclist(os.getcwd(),".bim",("_A1corr.bim","_noDup.bim"))
loc_list

EIRA detected!


['/exports/reum/CKe/Pipelinerun_withPC/EIRA/mergedChr/EIRA_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/RAMS/mergedChr/RAMS_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/Madrid/mergedChr/madrid_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/Manchester/mergedChr/Manchester_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/Tacera/Final_imputed/TACERA_HRCv1_1_FINAL.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/EARTH_B/mergedChr/EARTH_setB_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/Glasgow/mergedChr/Glasgow_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/ACPA/mergedChr/ACPAneg_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/SERA/mergedChr/sera_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/EARTH_A/mergedChr/EA127genotypes_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/PEAC/mergedChr/PEAC_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/Vienna/mergedChr/vienna_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/EAC/mergedChr/EAC_merged.bim']

In [35]:
# Leeds.rename(columns={"A1":"A1_Leeds"},inplace=True)
# merge_temp = Leeds.loc[:,['SNP','A1_Leeds']]
# cols_to_keep = ['SNP','A1_Leeds']
multiallelics = pd.DataFrame(columns=["SNP"])

In [37]:
for i in range(len(loc_list)):
    cohort_name = loc_list[i].split("/")[-3]
    print("reading cohort ",cohort_name)
    bim_cohort = pd.read_csv(loc_list[i],sep='\t',header=None) #nrows only for testing
    bim_cohort.rename(columns={0:"CHR",1:"SNP",3:"BP",4:"A1",5:"A2"},inplace=True)   
    if cohort_name != "Tacera": # convert snp id format to chr:bp
        bim_cohort['SNP']=bim_cohort['SNP'].str.split(":",expand=True)[0]+":"+bim_cohort['SNP'].str.split(":",expand=True)[1]
    multiallelics_inCohort = bim_cohort[bim_cohort.duplicated('SNP')]
    multiallelics = multiallelics.merge(multiallelics_inCohort,how="outer",on="SNP").loc[:,['SNP']]

reading cohort  EIRA
reading cohort  RAMS
reading cohort  Madrid
reading cohort  Manchester
reading cohort  Tacera
reading cohort  EARTH_B
reading cohort  Glasgow
reading cohort  ACPA
reading cohort  SERA
reading cohort  EARTH_A
reading cohort  PEAC
reading cohort  Vienna
reading cohort  EAC


In [39]:
# multiallelics.duplicated().sum()
multiallelics.drop_duplicates(inplace=True)
multiallelics['SNP'].to_csv("duplist_within.txt",sep='\t',index=False,header=False) 

0

In [48]:
bim_cohort = pd.read_csv(loc_list[0],sep='\t',header=None)
bim_cohort

,0,1,2,3,4,5
0,1,1:57183:A:G,0,57183,G,A
1,1,1:62055:G:C,0,62055,C,G
2,1,1:84139:A:T,0,84139,T,A
3,1,1:526561:T:G,0,526561,G,T
4,1,1:526692:C:T,0,526692,T,C
...,...,...,...,...,...,...
20510406,22,22:51237063:T:C,0,51237063,C,T
20510407,22,22:51237069:T:C,0,51237069,C,T
20510408,22,22:51238513:C:G,0,51238513,G,C
20510409,22,22:51239586:T:G,0,51239586,G,T


In [104]:
bim_cohort.rename(columns={0:"CHR",1:"SNP",3:"BP",4:"A1",5:"A2"},inplace=True) 
if cohort_name != "Tacera": # convert snp id format to chr:bp
    bim_cohort['SNP']=bim_cohort['SNP'].str.split(":",expand=True)[0]+":"+bim_cohort['SNP'].str.split(":",expand=True)[1]
bim_cohort

,CHR,SNP,2,BP,A1,A2,chrbp
0,1,1:54353,0,54353,A,C,1:54353
1,1,1:57183,0,57183,G,A,1:57183
2,1,1:62157,0,62157,A,G,1:62157
3,1,1:79137,0,79137,T,A,1:79137
4,1,1:85108,0,85108,C,T,1:85108
...,...,...,...,...,...,...,...
18801638,22,22:51236700,0,51236700,G,T,22:51236700
18801639,22,22:51237063,0,51237063,C,T,22:51237063
18801640,22,22:51237136,0,51237136,A,G,22:51237136
18801641,22,22:51239586,0,51239586,G,T,22:51239586


In [40]:
# duplicates_chrbp = pd.read_csv("duplicates_snp_chrbp.txt",sep='\t')
duplicates_chrbp = pd.read_csv("duplist_all.txt",sep='\t')
duplicates_chrbp

,SNP,Uniqueness
0,1:858801,AG
1,1:860416,GA
2,1:860688,GA
3,1:861630,GA
4,1:862866,CT
...,...,...
567768,10:97943604,GC
567769,10:121754888,GC
567770,15:60622072,AT
567771,19:31907554,AT


In [43]:
duplicates_within = pd.read_csv("duplist_within.txt",sep='\t',header=None)
duplicates_within

,0
0,1:788595
1,1:901922
2,1:1034498
3,1:1097335
4,1:1164538
...,...
73844,22:48017386
73845,22:49430136
73846,22:49802109
73847,22:49853669


In [46]:
bim_cohort

,CHR,SNP,2,BP,A1,A2
0,1,1:54353,0,54353,A,C
1,1,1:57183,0,57183,G,A
2,1,1:62157,0,62157,A,G
3,1,1:79137,0,79137,T,A
4,1,1:85108,0,85108,C,T
...,...,...,...,...,...,...
9995,1,1:2063841,0,2063841,G,C
9996,1,1:2063873,0,2063873,C,T
9997,1,1:2064088,0,2064088,C,T
9998,1,1:2064242,0,2064242,T,C


In [54]:
dup_within

,0_x,1,2,3,4,5,chrbp,0_y
0,1,1:788595:C:G,0,788595,G,C,1:788595,1:788595
1,1,1:788595:C:T,0,788595,T,C,1:788595,1:788595
2,1,1:901922:G:A,0,901922,A,G,1:901922,1:901922
3,1,1:901922:G:C,0,901922,C,G,1:901922,1:901922
4,1,1:966682:C:A,0,966682,A,C,1:966682,1:966682
...,...,...,...,...,...,...,...,...
93304,22,22:51085788:G:A,0,51085788,A,G,22:51085788,22:51085788
93305,22,22:51126242:C:A,0,51126242,A,C,22:51126242,22:51126242
93306,22,22:51144305:G:A,0,51144305,A,G,22:51144305,22:51144305
93307,22,22:51144305:G:C,0,51144305,C,G,22:51144305,22:51144305


In [88]:
def getsnpid(x):
    if x.loc[:,'1_x'].isnull() and x.loc[:,'1_y'].isnull():
        snp = x.loc[:'1_x']
    else:
        snp = x.loc[:,'1_x']+x.loc[:,'1_y']
    return snp

In [94]:
dup

,0,1_x,2_x,3_x,4_x,5_x,chrbp,SNP,Uniqueness,0_x,1_y,2_y,3_y,4_y,5_y,0_y,SNP_allele
0,1.0,1:715142:G:A,0.0,715142.0,A,G,1:715142,1:715142,TA,,,,,,,,1:715142:G:A
1,1.0,1:830181:A:G,0.0,830181.0,A,G,1:830181,1:830181,AG,,,,,,,,1:830181:A:G
2,1.0,1:831489:C:T,0.0,831489.0,C,T,1:831489,1:831489,CT,,,,,,,,1:831489:C:T
3,1.0,1:831909:C:T,0.0,831909.0,C,T,1:831909,1:831909,CT,,,,,,,,1:831909:C:T
4,1.0,1:832066:G:C,0.0,832066.0,G,C,1:832066,1:832066,GC,,,,,,,,1:832066:G:C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627283,,,,,,,22:50503336,,,22.0,22:50503336:C:T,0.0,50503336.0,T,C,22:50503336,22:50503336:C:T
627284,,,,,,,22:50996974,,,22.0,22:50996974:C:A,0.0,50996974.0,A,C,22:50996974,22:50996974:C:A
627285,,,,,,,22:51019113,,,22.0,22:51019113:A:C,0.0,51019113.0,C,A,22:51019113,22:51019113:A:C
627286,,,,,,,22:51144305,,,22.0,22:51144305:G:A,0.0,51144305.0,A,G,22:51144305,22:51144305:G:A


In [81]:
dup['SNP_allele'].duplicated().sum()

0

In [77]:
#generate list of duplicates snps for each cohort based on multiallelics across all cohorts
for run in loc_list:
    bim_cohort = pd.read_csv(run,sep='\t',header=None)
    bim_cohort['chrbp'] = bim_cohort[1].str.rsplit(":",n=2,expand=True)[0]
    dup_across = pd.merge(bim_cohort,duplicates_chrbp,how='inner',left_on="chrbp",right_on="SNP")
    dup_within = pd.merge(bim_cohort,duplicates_within,how='inner',left_on="chrbp",right_on=0)
    dup = pd.merge(dup_across,dup_within,how="outer",on="chrbp").fillna("")
    dup['SNP_allele'] = dup['1_x']+dup['1_y']
    new_loc = run.split(".")[0]+"_dupsExclude.txt"
    dup['SNP_allele'].to_csv(new_loc,sep='\t',index=False,header=False) #should be no duplicates any more
    print("saved dups for: ",os.path.split(new_loc)[1])

saved dups for:  EIRA_merged_dupsExclude.txt
saved dups for:  RAMS_merged_dupsExclude.txt
saved dups for:  madrid_merged_dupsExclude.txt
saved dups for:  Manchester_merged_dupsExclude.txt
saved dups for:  TACERA_HRCv1_1_FINAL_dupsExclude.txt
saved dups for:  EARTH_setB_merged_dupsExclude.txt
saved dups for:  Glasgow_merged_dupsExclude.txt
saved dups for:  ACPAneg_merged_dupsExclude.txt
saved dups for:  sera_merged_dupsExclude.txt
saved dups for:  EA127genotypes_merged_dupsExclude.txt
saved dups for:  PEAC_merged_dupsExclude.txt
saved dups for:  vienna_merged_dupsExclude.txt
saved dups for:  EAC_merged_dupsExclude.txt


In [82]:
#process leeds to remove dups/multiallelics
#Leeds = pd.read_csv("/exports/reum/CKe/Pipelinerun_withPC/Leeds/mergedChr/Leeds_noNA_maf_meta.txt",sep='\t')
# dup_across = pd.merge(Leeds,duplicates_chrbp,how='inner',left_on="SNP",right_on="SNP")
# dup_within = pd.merge(Leeds,duplicates_within,how='inner',left_on="SNP",right_on=0)
# dup = pd.merge(dup_across,dup_within,how="outer",on="").fillna("")
dup_within

,CHR,SNP,BP,A1_Leeds,A2,TEST,NMISS,OR,SE,BETA,L95,U95,STAT,P,0
0,1,1:1164538,1164538,T,T,ADD,563,1.2780,0.2650,0.245296,0.76020,2.148,0.92520,0.35480,1:1164538
1,1,1:1246438,1246438,A,A,ADD,561,1.4190,0.3484,0.349952,0.71700,2.810,1.00500,0.31480,1:1246438
2,1,1:2335969,2335969,G,G,ADD,556,1.1350,0.1385,0.126633,0.86520,1.489,0.91420,0.36060,1:2335969
3,1,1:2448561,2448561,A,A,ADD,562,0.7692,0.1762,-0.262404,0.54460,1.086,-1.48900,0.13640,1:2448561
4,1,1:3103312,3103312,T,T,ADD,551,0.9984,0.1592,-0.001601,0.73070,1.364,-0.01004,0.99200,1:3103312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15902,22,22:50957462,50957462,C,G,ADD,559,1.0950,0.1576,0.090754,0.80400,1.491,0.57550,0.56490,22:50957462
15903,22,22:51015210,51015210,G,T,ADD,562,1.3790,0.1374,0.321359,1.05300,1.805,2.33900,0.01933,22:51015210
15904,22,22:51019113,51019113,C,G,ADD,563,0.3832,1.1300,-0.959198,0.04183,3.510,-0.84880,0.39600,22:51019113
15905,22,22:51062201,51062201,T,C,ADD,560,0.3996,1.0880,-0.917291,0.04739,3.370,-0.84320,0.39910,22:51062201


In [25]:
for run in loc_list:
    corrlist = pd.read_csv(run,sep='\t',header=None)
    dups = corrlist.iloc[np.where(corrlist[3]==corrlist[4])[0],:]
    new_loc = run.split(".")[0]+"_dupAlleles_exclude.txt"
    dups[0].to_csv(new_loc,index=False,header=False)
    print("saved exclude list for ",run)

saved exclude list for  /exports/reum/CKe/Pipelinerun_withPC/EIRA/mergedChr/EIRA_merged_AllelesCorr.txt
saved exclude list for  /exports/reum/CKe/Pipelinerun_withPC/RAMS/mergedChr/RAMS_merged_AllelesCorr.txt
saved exclude list for  /exports/reum/CKe/Pipelinerun_withPC/Madrid/mergedChr/madrid_merged_AllelesCorr.txt
saved exclude list for  /exports/reum/CKe/Pipelinerun_withPC/Manchester/mergedChr/Manchester_merged_AllelesCorr.txt
saved exclude list for  /exports/reum/CKe/Pipelinerun_withPC/Tacera/Final_imputed/TACERA_HRCv1_1_FINAL_AllelesCorr.txt
saved exclude list for  /exports/reum/CKe/Pipelinerun_withPC/EARTH_B/mergedChr/EARTH_setB_merged_AllelesCorr.txt
saved exclude list for  /exports/reum/CKe/Pipelinerun_withPC/Glasgow/mergedChr/Glasgow_merged_AllelesCorr.txt
saved exclude list for  /exports/reum/CKe/Pipelinerun_withPC/ACPA/mergedChr/ACPAneg_merged_AllelesCorr.txt
saved exclude list for  /exports/reum/CKe/Pipelinerun_withPC/SERA/mergedChr/sera_merged_AllelesCorr.txt
saved exclude l

In [118]:
compareA1 = pd.read_csv("compareA1_stat.txt",sep='\t',nrows=100000,usecols=['SNP','Uniqueness'])
compareA2 = pd.read_csv("compareA2_stat.txt",sep='\t',nrows=100000,usecols=['SNP','Uniqueness'])

In [218]:
cleanA1 = compareA1.groupby("SNP",as_index=False).agg(list)
cleanA2 = compareA2.groupby("SNP",as_index=False).agg(list)

In [119]:
compareA1

,SNP,Uniqueness
0,1:729679,C
1,1:752566,G
2,1:752721,A
3,1:753405,C
4,1:754182,A
...,...,...
99995,1:186940756,A
99996,1:186941600,G
99997,1:186942813,C
99998,1:186943559,A


In [219]:
cleanA1

,SNP,Uniqueness
0,1:10000400,[T]
1,1:100011667,[T]
2,1:1000156,[C]
3,1:100015703,[C]
4,1:100021743,[C]
...,...,...
75653,1:99991628,[T]
75654,1:99992311,[G]
75655,1:99993077,[C]
75656,1:9999539,[A]


In [220]:
def getstr(a):
    a = "".join(set(a))
    a = "".join(set(a))
    return a

In [221]:
cleanA1['strUniqueness'] = cleanA1['Uniqueness'].apply(getstr)
cleanA2['strUniqueness'] = cleanA2['Uniqueness'].apply(getstr)

In [204]:
list_unique_test = cleanA1['Uniqueness'][42290]
string_test = getstr(list_unique_test)
string_test

'CA'

In [214]:
lens = A1A2['count_UniquenessPerSNP']
dict_count = {}
unique_keys = lens.unique()
for i in unique_keys:
    dict_count[i] = (lens==i).sum()

In [215]:
dict_count

{1: 60775, 2: 14857, 3: 26}

In [23]:
compareA2

,SNP,Uniqueness
0,1:729679,G
1,1:752566,A
2,1:752721,G
3,1:753405,A
4,1:754182,G
...,...,...
99995,1:186940756,T
99996,1:186941600,A
99997,1:186942813,G
99998,1:186943559,G


In [222]:
cleanA2

,SNP,Uniqueness,strUniqueness
0,1:10000400,[A],A
1,1:100011667,[C],C
2,1:1000156,[T],T
3,1:100015703,[T],T
4,1:100021743,[T],T
...,...,...,...
75653,1:99991628,[C],C
75654,1:99992311,[T],T
75655,1:99993077,[T],T
75656,1:9999539,[C],C


In [24]:
compareA2.duplicated("SNP").sum()

24342

In [227]:
A1A2 = pd.merge(cleanA1,cleanA2,how='inner',on="SNP",suffixes=['_A1','_A2'])
A1A2

,SNP,Uniqueness_A1,strUniqueness_A1,Uniqueness_A2,strUniqueness_A2
0,1:10000400,[T],T,[A],A
1,1:100011667,[T],T,[C],C
2,1:1000156,[C],C,[T],T
3,1:100015703,[C],C,[T],T
4,1:100021743,[C],C,[T],T
...,...,...,...,...,...
75653,1:99991628,[T],T,[C],C
75654,1:99992311,[G],G,[T],T
75655,1:99993077,[C],C,[T],T
75656,1:9999539,[A],A,[C],C


In [236]:
A1A2['UniquenessPerSNP'] = (A1A2['strUniqueness_A1']+A1A2['strUniqueness_A2']).map(getstr)
A1A2['count_UniquenessPerSNP'] = A1A2['UniquenessPerSNP'].map(len)

In [239]:
A1A2[A1A2['count_UniquenessPerSNP']>2]

,SNP,Uniqueness_A1,strUniqueness_A1,Uniqueness_A2,strUniqueness_A2,UniquenessPerSNP,count_UniquenessPerSNP
115,1:100154055,"[TC, TC, TC, TC, TC, TC, TC, T]",TC,"[GT, GT, GT, GT, GT, GT, GT, G]",TG,TCG,3
4591,1:107233800,"[G, GT, GT, GT, GT, GT, GT, GT, GT, GT, GT, GT...",TG,"[A, AG, AG, AG, AG, AG, AG, AG, AG, AG, AG, AG...",AG,TAG,3
4614,1:107262172,"[G, GT, GT, GT, GT, GT, GT, GT, GT, GT, GT, GT...",TG,"[C, CG, CG, CG, CG, CG, CG, CG, CG, CG, CG, CG...",CG,TCG,3
4784,1:107603330,"[CA, CA, CA, CA, CA, CA, CA, CA, CA, CA, CA, C...",CA,"[GC, GC, GC, GC, GC, GC, GC, GC, GC, GC, GC, G...",CG,CGA,3
4860,1:107724319,"[GC, GC, GC, GC, GC, GC, GC, GC, GC, GC, GC, G...",CG,"[TG, TG, TG, TG, TG, TG, TG, TG, TG, TG, TG, T...",TG,TCG,3
...,...,...,...,...,...,...,...
73958,1:96673400,"[TA, TA, TA, TA, TA, TA, TA, TA, TA, TA, TA, T...",TA,"[GT, GT, GT, GT, GT, GT, GT, GT, GT, GT, GT, G...",TG,TAG,3
74955,1:98910612,"[CA, CA, CA, CA, CA, CA, CA, CA, CA, CA, CA, C...",CA,"[GC, GC, GC, GC, GC, GC, GC, GC, GC, GC, GC, G...",CG,CGA,3
75031,1:98975907,"[GA, GA, GA, GA, GA, GA, GA, GA, GA, GA, GA, G...",AG,"[TG, TG, TG, TG, TG, TG, TG, TG, TG, TG, TG, T...",TG,TAG,3
75134,1:99054260,"[TA, TA, TA, TA, TA, TA, TA, TA, TA, TA, TA, T...",TA,"[CT, CT, CT, CT, CT, CT, CT, CT, CT, CT, CT, C...",TC,TAC,3


In [63]:
merge_a1 = pd.read_csv("compareA1.txt",sep='\t',index_col=0)
merge_a2 = pd.read_csv("compareA2.txt",sep='\t',index_col=0)
merge_a1.fillna("NotAvailable",inplace=True)
merge_a2.fillna("NotAvailable",inplace=True)

/home/cke/.conda/envs/CKe/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/cke/.conda/envs/CKe/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [88]:
dict_final_a1 = {}
dict_final_a2 = {}
for i in range(merge_a1.shape[1]-2,0,-1): #if there's uniqueness col, start from merge_temp.shape[1]-2 in range. If with NMISS, ends at 1
# make sure EIRA is the second last and Leeds is the last dict to be updated to final 
    dict_temp_a1 = merge_a1[merge_a1.iloc[:,i]!="NotAvailable"].iloc[:,[0,i]].set_index('SNP')
    dict_temp_a2 = merge_a2[merge_a2.iloc[:,i]!="NotAvailable"].iloc[:,[0,i]].set_index('SNP')
    name_a1 = dict_temp_a1.columns.tolist()[0]
    name_a2 = dict_temp_a2.columns.tolist()[0]
    print("updating dict ",name_a1,name_a2)
    dict_temp_a1 = dict_temp_a1.to_dict('dict')[name_a1]
    dict_temp_a2 = dict_temp_a2.to_dict('dict')[name_a2]
    #dict_final = dict_final | dict_temp  # new function to merge dicts in python3.9
    dict_final_a1.update(dict_temp_a1)
    dict_final_a2.update(dict_temp_a2)
    commonpairs(dict_final_a1,dict_final_a2,max(len(dict_final_a1),len(dict_final_a2)))
print("get both dict_final!")
#np.save("final_dict_A1.npy",dict_final_a1)
#np.save("final_dict_A2.npy",dict_final_a2)


updating dict  A1_EAC A2_EAC
breaking!
updating dict  A1_Vienna A2_Vienna
breaking!
updating dict  A1_PEAC A2_PEAC
breaking!
updating dict  A1_EARTH_A A2_EARTH_A


KeyboardInterrupt: 

In [85]:
def commonpairs(dict_final_a1,dict_final_a2,stop=1000):
    common_pairs = dict()
    count = 0
    for key in dict_final_a1:
        if (key in dict_final_a2 and dict_final_a1[key] == dict_final_a2[key]):
            common_pairs[key] = dict_final_a1[key]
            print(key,common_pairs[key])
        count += 1
        if count == stop:
            print("breaking!")
            break
    return common_pairs

In [87]:
commonpairs(dict_final_a1,dict_final_a2,len(dict_final_a1))

breaking!


{}

In [2]:
def getloclist(wd,end,filter_out):
    loc_list = []
    EIRA_found = False
    for root, dirs, files in os.walk(wd):
        for file in files:
            if file.endswith(end) and not file.endswith(filter_out):
                if root.split("/")[-2]=="EIRA":
                    loc = os.path.join(root, file)
                    loc_list.insert(0,loc)
                    EIRA_found = True
                else:
                    loc = os.path.join(root, file)
                    loc_list.append(loc) #get bim file location
    if EIRA_found == True:
        print("EIRA detected!")
    else:
        print("Warning! No EIRA bim detected! Use first scanned cohort as ref!")
    return loc_list

In [18]:
def get_unique(x): # which alleles are used in all cohorts for a snp
    u = list(x.unique())
    while "NotAvailable" in u:
        u.remove("NotAvailable")
    u = "".join(map(str,u))
    return u

In [9]:
def compareA1(cwd=os.getcwd(),out="compareA1.txt",test=False):
    #detects prepared inputs of METAL in cwd and generate a table comparing A1s used
    loc_list = getloclist(os.getcwd(),".bim",("_A1corr.bim","_noDup.bim"))
    if test:
        loc_list = loc_list[2:4] # only Madrid and Tacera
        print(loc_list)
    #use leeds as primary ref
    Leeds = pd.read_csv("/exports/reum/CKe/Pipelinerun_withPC/Leeds/mergedChr/Leeds_noNA_maf_meta.txt",sep='\t')
    Leeds.rename(columns={"A1":"A1_Leeds"},inplace=True)
    merge_temp = Leeds.loc[:,['SNP','A1_Leeds']]
    cols_to_keep = ['SNP','A1_Leeds']
    for i in range(len(loc_list)):
        cohort_name = loc_list[i].split("/")[-3]
        print("reading cohort ",cohort_name)
        bim_cohort = pd.read_csv(loc_list[i],sep='\t',header=None)
        bim_cohort.rename(columns={0:"CHR",1:"SNP",3:"BP",4:"A1",5:"A2"},inplace=True)   
        if cohort_name != "Tacera": # convert snp id format to chr:bp
            bim_cohort['SNP']=bim_cohort['SNP'].str.split(":",expand=True)[0]+":"+bim_cohort['SNP'].str.split(":",expand=True)[1]
        multiallelics_inCohort = bim_cohort.duplicated('SNP')
        bim_cohort.drop_duplicates('SNP', inplace=True) #keep first 
        merge_temp = pd.merge(merge_temp,bim_cohort,how='outer',on="SNP") 
        cols_to_keep.append('A1')
        merge_temp = merge_temp.loc[:,cols_to_keep]
        merge_temp = merge_temp.rename({'A1':'A1_'+cohort_name},axis=1)
        cols_to_keep = merge_temp.columns.tolist()
        print("cols to keep: ",cols_to_keep)
        #merge_temp.to_csv("/exports/reum/CKe/"+cohort1_name+"_compareA1.txt",sep='\t') #save after done with each cohort
    merge_temp['Uniqueness'] = "NotAvailable"
    merge_temp.to_csv(out,sep='\t')
    print("saved initial table!")
    return merge_temp

In [14]:
def compareA2(cwd=os.getcwd(),out="compareA2.txt",test=False):
    #detects prepared inputs of METAL in cwd and generate a table comparing A1s used
    loc_list = getloclist(cwd,".bim","_A1corr.bim")
    if test:
        loc_list = loc_list[2:4] # only Madrid and Tacera
        print(loc_list)
    #use leeds as primary ref
    Leeds = pd.read_csv("/exports/reum/CKe/Pipelinerun_withPC/Leeds/mergedChr/Leeds_noNA_maf_meta.txt",sep='\t')
    Leeds.rename(columns={"A2":"A2_Leeds"},inplace=True)
    merge_temp = Leeds.loc[:,['SNP','A2_Leeds']]
    cols_to_keep = ['SNP','A2_Leeds']
    for i in range(len(loc_list)):
        cohort_name = loc_list[i].split("/")[-3]
        print("reading cohort ",cohort_name)
        bim_cohort = pd.read_csv(loc_list[i],sep='\t',header=None)
        bim_cohort.rename(columns={0:"CHR",1:"SNP",3:"BP",4:"A1",5:"A2"},inplace=True)   
        if cohort_name != "Tacera": # convert snp id format to chr:bp
            bim_cohort['SNP']=bim_cohort['SNP'].str.split(":",expand=True)[0]+":"+bim_cohort['SNP'].str.split(":",expand=True)[1]
        #bim_cohort.drop_duplicates('SNP', keep = False, inplace=True)
        merge_temp = pd.merge(merge_temp,bim_cohort,how='outer',on="SNP") 
        cols_to_keep.append('A2')
        merge_temp = merge_temp.loc[:,cols_to_keep]
        merge_temp = merge_temp.rename({'A2':'A2_'+cohort_name},axis=1)
        cols_to_keep = merge_temp.columns.tolist()
        print("cols to keep: ",cols_to_keep)
        #merge_temp.to_csv("/exports/reum/CKe/"+cohort1_name+"_compareA1.txt",sep='\t') #save after done with each cohort
    merge_temp['Uniqueness'] = "NotAvailable"
    merge_temp.to_csv(out,sep='\t')
    print("saved initial table!")
    return merge_temp

EIRA detected!
['/exports/reum/CKe/Pipelinerun_withPC/Madrid/mergedChr/madrid_merged.bim', '/exports/reum/CKe/Pipelinerun_withPC/Manchester/mergedChr/Manchester_merged.bim']
reading cohort  Madrid


KeyboardInterrupt: 

In [10]:
def getUnique(compareFile="compareA1.txt",out="compareA1_stat.txt"):
    merge_temp = pd.read_csv(compareFile,sep='\t',index_col=0)
    merge_temp.fillna("NotAvailable",inplace=True)
    list_unique = merge_temp['Uniqueness'].tolist()
    merge_temp.fillna("NotAvailable",inplace=True)
    for index,row in merge_temp.iterrows():
        list_unique[index] = get_unique(row[1:-1])
    merge_temp['Uniqueness'] = list_unique
    merge_temp.to_csv(out,sep='\t')
    print("saved with uniqueness")
    # count how consistent the use of effect allele for each snp
    list_u = list_unique
    dict_count = {}
    for i in range(len(list_u)):
        list_u[i] = list_u[i].replace("nan","")
        if len(list_u[i]) in dict_count:
            dict_count[len(list_u[i])] += 1
        else:
            dict_count[len(list_u[i])] = 1
    return dict_count

In [16]:
getUnique("compareA2.txt","compareA2_stat.txt")

/tmp/ipykernel_4192516/472004495.py:1: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  getUnique("compareA2.txt","compareA2_stat.txt")
/home/cke/.conda/envs/CKe/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


KeyboardInterrupt: 

In [23]:
dict_final_a1 = np.load("/exports/reum/CKe/Pipelinerun_withPC/final_dict_A1.npy",allow_pickle=True)
dict_final_a2 = np.load("/exports/reum/CKe/Pipelinerun_withPC/final_dict_A2.npy",allow_pickle=True)

dict_final_a1 = dict_final_a1.item()
dict_final_a2 = dict_final_a2.item()

In [65]:
dict_final_a1['1:753541']

'A'

In [66]:
dict_final_a2['1:753541']

'A'

In [70]:
bim_overlap = pd.read_csv("/exports/reum/CKe/PCArun/Tacera_onRef/Merge_cohorts/cohorts_merged.bim",sep="\t",header=None)
bim_overlap

,0,1,2,3,4,5
0,1,1:909309:T:C,0,909309,C,T
1,1,1:935398:G:A,0,935398,A,G
2,1,1:944564:T:C,0,944564,C,T
3,1,1:962210:A:G,0,962210,G,A
4,1,1:1023310:T:C,0,1023310,C,T
...,...,...,...,...,...,...
223462,22,22:51219006:G:A,0,51219006,A,G
223463,22,22:51219704:G:A,0,51219704,A,G
223464,22,22:51222100:G:T,0,51222100,T,G
223465,22,22:51229805:T:C,0,51229805,C,T


In [30]:
bim_cohort.rename(columns={0:"CHR",1:"SNP",3:"BP",4:"A1",5:"A2"},inplace=True)   
bim_cohort['SNP']=bim_cohort['SNP'].str.split(":",expand=True)[0]+":"+bim_cohort['SNP'].str.split(":",expand=True)[1]
#bim_cohort.drop_duplicates('SNP', keep = False, inplace=True) #move to further step
bim_cohort['A1'] = bim_cohort['SNP'].map(dict_final) 
new_loc = loc_list[1].split(".")[0]+"_A1corr.txt"
bim_cohort['A1'].to_csv(new_loc,sep='\t',index=False,header=False)
print("work done on cohort ",cohort_name," count: ",bim_cohort.shape[0])

NameError: name 'cohort_name' is not defined

In [59]:
update_table = bim_cohort.loc[:,[1,4,5]]
update_table['A1corr'] = bim2[4]
update_table['A2corr'] = bim2[5]

In [65]:
update_table.iloc[994,]

1         1:959231:G:A
4                    A
5                    G
A1corr               G
A2corr               G
Name: 994, dtype: object

In [61]:
update_table.to_csv("/exports/reum/CKe/Pipelinerun_withPC/RAMS/mergedChr/RAMS_AllelesCorr.txt",sep='\t',index=False,header=False)

In [62]:
def unifyA1(cwd=os.getcwd(),compareFile="compareA1.txt"):
# make sure all snps across different cohorts use the same A1 effect allele (minor)
    merge_temp = pd.read_csv(compareFile,sep='\t',index_col=0)
    merge_temp.fillna("NotAvailable",inplace=True)
    #merge_temp.drop(['Unnamed: 0'],axis=1,inplace=True)
    dict_final = {}
    for i in range(merge_temp.shape[1]-2,0,-1): #if there's uniqueness col, start from merge_temp.shape[1]-2 in range. If with NMISS, ends at 1
    # make sure EIRA is the second last and Leeds is the last dict to be updated to final 
        dict_temp = merge_temp[merge_temp.iloc[:,i]!="NotAvailable"].iloc[:,[0,i]].set_index('SNP')
        name = dict_temp.columns.tolist()[0]
        dict_temp = dict_temp.to_dict('dict')[name]
        print("updating dict ",name)
        #dict_final = dict_final | dict_temp  # new function to merge dicts in python3.9
        dict_final.update(dict_temp)
    print("get A1 dict_final!")
    np.save("final_dict_A1.npy",dict_final)
    return dict_final
    
def unifyA2(cwd=os.getcwd(),compareFile="compareA2.txt"):
# make sure all snps across different cohorts use the same A1 effect allele (minor)
    merge_temp = pd.read_csv(compareFile,sep='\t',index_col=0)
    merge_temp.fillna("NotAvailable",inplace=True)
    #merge_temp.drop(['Unnamed: 0'],axis=1,inplace=True)
    dict_final = {}
    for i in range(merge_temp.shape[1]-2,0,-1): #if there's uniqueness col, start from merge_temp.shape[1]-2 in range. If with NMISS, ends at 1
    # make sure EIRA is the second last and Leeds is the last dict to be updated to final 
        dict_temp = merge_temp[merge_temp.iloc[:,i]!="NotAvailable"].iloc[:,[0,i]].set_index('SNP')
        name = dict_temp.columns.tolist()[0]
        dict_temp = dict_temp.to_dict('dict')[name]
        print("updating dict ",name)
        #dict_final = dict_final | dict_temp  # new function to merge dicts in python3.9
        dict_final.update(dict_temp)
    print("get A2 dict_final!")
    np.save("final_dict_A2.npy",dict_final)
    return dict_final

def updateAlleles(dict_a1,dict_a2,cwd=os.getcwd()):
    loc_list = getloclist(cwd,".bim","_A1corr.bim")
    for i in range(len(loc_list)):
        cohort_name = loc_list[i].split("/")[-3]
        print("reading cohort ",cohort_name)
        bim_cohort = pd.read_csv(loc_list[i],sep='\t',header=None)
        bim_cohort.rename(columns={0:"CHR",1:"SNP",3:"BP",4:"A1",5:"A2"},inplace=True)   
        update_table = bim_cohort.loc[:,['SNP','A1','A2']]
        if cohort_name != "Tacera": # convert snp id format to chr:bp
            bim_cohort['SNP']=bim_cohort['SNP'].str.split(":",expand=True)[0]+":"+bim_cohort['SNP'].str.split(":",expand=True)[1]
        #bim_cohort.drop_duplicates('SNP', keep = False, inplace=True) #move to further step
        update_table['corrA1'] = bim_cohort['SNP'].map(dict_a1)
        update_table['corrA2'] = bim_cohort['SNP'].map(dict_a2)
        new_loc = loc_list[i].split(".")[0]+"_AllelesCorr.txt"
        update_table.to_csv(new_loc,sep='\t',index=False,header=False)
        print("work done on cohort ",cohort_name," count: ",bim_cohort.shape[0])

In [48]:
dict_final_a1 = unifyA1()
dict_final_a2 = unifyA2()
updateAlleles(dict_final_a1,dict_final_a2)

KeyboardInterrupt: 

In [32]:
loc_list = []
for root, dirs, files in os.walk(os.getcwd()):
    for file in files:
        if file.endswith('_AllelesCorr.txt'): #should be changed when in new file structure
            loc_list.append(root)
loc_list

['/exports/reum/CKe/Pipelinerun_withPC/RAMS/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/Madrid/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/Manchester/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/Tacera/Final_imputed',
 '/exports/reum/CKe/Pipelinerun_withPC/EARTH_B/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/EIRA/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/Glasgow/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/ACPA/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/SERA/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/EARTH_A/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/PEAC/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/Vienna/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/EAC/mergedChr']

In [9]:
#merge_temp = pd.read_csv("compareA1.txt",sep='\t',index_col=0)
#merge_temp.fillna("NotAvailable",inplace=True)
merge_temp

,SNP,A1_Leeds,A1_EIRA,A1_RAMS,A1_Madrid,A1_Tacera,A1_EARTH_B,A1_ACPA,A1_SERA,A1_EARTH_A,A1_PEAC,A1_Vienna,A1_EAC,Uniqueness
0,1:729679,C,C,C,C,NotAvailable,C,C,C,C,NotAvailable,C,C,NotAvailable
1,1:752566,G,G,G,G,NotAvailable,G,G,G,G,NotAvailable,G,G,NotAvailable
2,1:752721,A,A,A,A,NotAvailable,A,A,A,A,NotAvailable,A,A,NotAvailable
3,1:753405,C,C,C,C,NotAvailable,C,C,C,C,NotAvailable,C,C,NotAvailable
4,1:753541,A,A,A,A,NotAvailable,A,A,A,A,A,A,A,NotAvailable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36047201,22:51143876,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,C,NotAvailable
36047202,22:51151905,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,C,NotAvailable
36047203,22:51157099,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,T,NotAvailable
36047204,22:51157493,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,NotAvailable,T,NotAvailable


In [24]:
dict_temp = merge_temp[merge_temp.iloc[:,10]!="NotAvailable"].iloc[:,[0,10]].set_index('SNP')
name = dict_temp.columns.tolist()[0]
dict_temp = dict_temp.to_dict('dict')[name]

In [25]:
#dict_final = {}
dict_final.update(dict_temp)

In [26]:
dict_final

{'1:729679': 'C',
 '1:752566': 'G',
 '1:752721': 'A',
 '1:753405': 'C',
 '1:753541': 'A',
 '1:754182': 'A',
 '1:754192': 'A',
 '1:754334': 'T',
 '1:754503': 'G',
 '1:754964': 'C',
 '1:755775': 'A',
 '1:755890': 'A',
 '1:756604': 'A',
 '1:757640': 'G',
 '1:757734': 'C',
 '1:757936': 'C',
 '1:758144': 'A',
 '1:758626': 'C',
 '1:760912': 'C',
 '1:761147': 'T',
 '1:761732': 'C',
 '1:766007': 'C',
 '1:769223': 'G',
 '1:771823': 'T',
 '1:771967': 'A',
 '1:772755': 'A',
 '1:777122': 'A',
 '1:778745': 'G',
 '1:779322': 'G',
 '1:780785': 'T',
 '1:782981': 'T',
 '1:785050': 'G',
 '1:785989': 'T',
 '1:787606': 'T',
 '1:787844': 'C',
 '1:791191': 'A',
 '1:797440': 'C',
 '1:798400': 'G',
 '1:798959': 'A',
 '1:801467': 'C',
 '1:804759': 'T',
 '1:808631': 'G',
 '1:808928': 'C',
 '1:845635': 'T',
 '1:845938': 'A',
 '1:846078': 'T',
 '1:846398': 'A',
 '1:846808': 'T',
 '1:846864': 'C',
 '1:847228': 'T',
 '1:847491': 'A',
 '1:848090': 'A',
 '1:848445': 'A',
 '1:848456': 'G',
 '1:848738': 'T',
 '1:850062

In [28]:
dict_final['1:532005']

'T'

In [7]:
for i in range(merge_temp.shape[1]-2,0,-1): #if there's uniqueness col, start from merge_temp.shape[1]-2 in range. If with NMISS, ends at 1
    # make sure EIRA is the second last and Leeds is the last dict to be updated to final 
    dict_temp = merge_temp[merge_temp.iloc[:,i]!="NotAvailable"].iloc[:,[0,i]].set_index('SNP')
    name = dict_temp.columns.tolist()[0]
    dict_temp = dict_temp.to_dict('dict')[name]
    print("updating dict ",name)
    #dict_final = dict_final | dict_temp  # new function to merge dicts in python3.9
    dict_final.update(dict_temp)
print("get dict_final!")

,SNP,A1_Leeds,A1_EIRA,A1_RAMS,A1_Madrid,A1_Tacera,A1_EARTH_B,A1_ACPA,A1_SERA,A1_EARTH_A,A1_PEAC,A1_Vienna,A1_EAC,Uniqueness
22620118,1:754758,NotAvailable,NotAvailable,NotAvailable,NotAvailable,G,NotAvailable,NotAvailable,NotAvailable,NotAvailable,G,NotAvailable,G,NotAvailable


In [25]:
# copy files from cohorts dir to generic-metal for meta-analysis
# can also be achieved by command:   
# find . -iname *_maf_meta.txt | xargs -i cp {} "/exports/reum/CKe/generic-metal/RunMeta/"
def autoCopyPaste(end='_noNA_maf_meta.txt',cwd=os.getcwd(),to="/exports/reum/CKe/generic-metal/RunMeta/"):
    loc_list=[]
    for root, dirs, files in os.walk(cwd):
        for file in files:
            if file.endswith(end):
                loc = os.path.join(root, file)
                loc_list.append(loc)
    for i in loc_list:
        os.system("cp "+i+" "+to)
    os.system("mv "+mainwk+"/RunMeta/"+"EARTH_A_noNA_maf_meta.txt "+mainwk+"/RunMeta/"+"EARTHA_noNA_maf_meta.txt")
    os.system("mv "+mainwk+"/RunMeta/"+"EARTH_B_noNA_maf_meta.txt "+mainwk+"/RunMeta/"+"EARTHB_noNA_maf_meta.txt") #WHY WOULD ANYONE INCLUDE A DAMN UNDERSCORE IN THEIR COHORT NAME, DON'T DO THIS ANYMORE!

In [26]:
def run_GWAS(cwd=os.getcwd()):
# run gwas script in each cohort
    loc_list = []
    for root, dirs, files in os.walk(cwd):
        for file in files:
            if file.endswith('.bed'):
                loc_list.append(root)
    for run in loc_list:
        os.system(run+"/Run_gwas.sh")

In [30]:
loc_list = []
for root, dirs, files in os.walk(os.getcwd()):
    for file in files:
        if file.endswith('_AllelesCorr.txt'):
            loc_list.append(root)
loc_list

['/exports/reum/CKe/Pipelinerun_withPC/RAMS/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/Madrid/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/Manchester/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/Tacera/Final_imputed',
 '/exports/reum/CKe/Pipelinerun_withPC/EARTH_B/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/EIRA/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/Glasgow/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/ACPA/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/SERA/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/EARTH_A/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/PEAC/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/Vienna/mergedChr',
 '/exports/reum/CKe/Pipelinerun_withPC/EAC/mergedChr']

In [244]:
loc_list = getloclist(os.getcwd(),"noNA.assoc.logistic"," ")
loc_list

EIRA detected!


['/exports/reum/CKe/Pipelinerun_withPC/EIRA/mergedChr/noNA.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/RAMS/mergedChr/noNA.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/Madrid/mergedChr/noNA.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/Manchester/mergedChr/noNA.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/Tacera/Final_imputed/noNA.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/EARTH_B/mergedChr/noNA.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/Glasgow/mergedChr/noNA.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/ACPA/mergedChr/noNA.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/SERA/mergedChr/noNA.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/EARTH_A/mergedChr/noNA.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/PEAC/mergedChr/noNA.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/Vienna/mergedChr/noNA.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/EAC/mergedChr/noNA.assoc

In [265]:
# summary_origin = pd.read_csv("/exports/reum/CKe/Pipelinerun_withPC/Leeds/mergedChr/Leeds_noNA_maf_meta.txt",sep='\t')
# SNP = summary_origin['SNP'].str.split(':',expand=True)[0]+":"+summary_origin['SNP'].str.split(':',expand=True)[1]+":"+summary_origin['A1']
# summary_origin['SNP'] = SNP
summary_origin.to_csv("/exports/reum/CKe/Pipelinerun_withPC/Leeds/mergedChr/Leeds_noNA_maf_meta.txt",sep='\t',index=False)

In [264]:
summary_origin['SNP'].duplicated().sum()

0

In [298]:
os.chdir("..")

In [299]:
os.getcwd()

'/exports/reum/CKe/Pipelinerun_withPC'

In [18]:
def meta_prepare(cwd=os.getcwd(), key="noNA.assoc.logistic", skip_key=" ", out_suffix="_noNA_maf_meta.txt",cohort_skip=[]):
# list of cohorts could be skipped:['EIRA', 'RAMS', 'Madrid', 'Manchester', 'Tacera', 'EARTH_B', 'Glasgow', 'ACPA', 'SERA', 'EARTH_A', 'PEAC', 'Vienna', 'EAC']
# detect noNA.assoc.logistic files and prepare them for meta-analysis
# get correct Minor allele a1, calculate BETA from OR, drop all biallelic snps
# to upgrade:exclude certain cohort
# Leeds is processed separately 
    loc_list = getloclist(os.getcwd(),key,skip_key)
    print(loc_list)
    for run in loc_list: 
        cohort_name = run.split("/")[-3]
        if cohort_name not in cohort_skip:
            summary_origin = pd.read_csv(run,sep='\s+')
            BETA = np.log(summary_origin['OR'])
            summary_origin.insert(8,'BETA',BETA)
            SNP = summary_origin['SNP'].str.split(':',expand=True)[0]+":"+summary_origin['SNP'].str.split(':',expand=True)[1]+":"+summary_origin['A1']
            summary_origin['SNP'] = SNP
            if summary_origin['SNP'].duplicated().sum() != 0:
                print("duplicate snp detected in summary stat file! check for multiallelics?")
            summary_origin.drop_duplicates('SNP', keep = False, inplace=True)
            new_name = cohort_name + out_suffix
            new_loc = os.path.join(os.path.split(run)[0],new_name)
            summary_origin.to_csv(new_loc,sep='\t',index=0)
            print(cohort_name+": ",summary_origin.shape[0],"saved to: ",new_loc)
        else:
            print("skipping cohort",cohort_name)

In [11]:
meta_prepare(out_suffix="_noNA_noMAFfilter_meta.txt",key="noNA_noMAFfilter.assoc.logistic",cohort_skip=['EIRA', 'RAMS', 'Madrid', 'Manchester', 'Tacera', 'EARTH_B', 'SERA', 'EARTH_A', 'PEAC', 'Vienna', 'EAC'])

EIRA detected!
skipping cohort EIRA
skipping cohort RAMS
skipping cohort Madrid
skipping cohort Manchester
skipping cohort Tacera
skipping cohort EARTH_B
Glasgow:  5875655 saved to:  /exports/reum/CKe/Pipelinerun_withPC/Glasgow/mergedChr/Glasgow_noNA_noMAFfilter_meta.txt
ACPA:  4691555 saved to:  /exports/reum/CKe/Pipelinerun_withPC/ACPA/mergedChr/ACPA_noNA_noMAFfilter_meta.txt
skipping cohort SERA
skipping cohort EARTH_A
skipping cohort PEAC
skipping cohort Vienna
skipping cohort EAC


In [20]:
meta_prepare(out_suffix="_noNA_noMAFfilter_MTX_meta.txt",key="noNA_noMAFfilter_MTX.assoc.logistic")

Warning! No EIRA bim detected! Use first scanned cohort as ref!
['/exports/reum/CKe/Pipelinerun_withPC/RAMS/mergedChr/RAMS_noNA_noMAFfilter_MTX.assoc.logistic', '/exports/reum/CKe/Pipelinerun_withPC/Madrid/mergedChr/Madrid_noNA_noMAFfilter_MTX.assoc.logistic', '/exports/reum/CKe/Pipelinerun_withPC/Tacera/Final_imputed/Tacera_noNA_noMAFfilter_MTX.assoc.logistic', '/exports/reum/CKe/Pipelinerun_withPC/EARTH_B/mergedChr/EARTHB_noNA_noMAFfilter_MTX.assoc.logistic', '/exports/reum/CKe/Pipelinerun_withPC/Glasgow/mergedChr/Glasgow_noNA_noMAFfilter_MTX.assoc.logistic', '/exports/reum/CKe/Pipelinerun_withPC/SERA/mergedChr/SERA_noNA_noMAFfilter_MTX.assoc.logistic', '/exports/reum/CKe/Pipelinerun_withPC/EARTH_A/mergedChr/EARTHA_noNA_noMAFfilter_MTX.assoc.logistic', '/exports/reum/CKe/Pipelinerun_withPC/Vienna/mergedChr/Vienna_noNA_noMAFfilter_MTX.assoc.logistic']
RAMS:  100 saved to:  /exports/reum/CKe/Pipelinerun_withPC/RAMS/mergedChr/RAMS_noNA_noMAFfilter_MTX_meta.txt
Madrid:  98 saved to:  /ex

In [14]:
loc_list = getloclist(os.getcwd(),"noNA_noMAFfilter.assoc.logistic"," ")
loc_list

EIRA detected!


['/exports/reum/CKe/Pipelinerun_withPC/EIRA/mergedChr/noNA_noMAFfilter.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/RAMS/mergedChr/noNA_noMAFfilter.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/Madrid/mergedChr/noNA_noMAFfilter.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/Manchester/mergedChr/noNA_noMAFfilter.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/Tacera/Final_imputed/noNA_noMAFfilter.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/EARTH_B/mergedChr/noNA_noMAFfilter.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/ACPA/mergedChr/noNA_noMAFfilter.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/SERA/mergedChr/noNA_noMAFfilter.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/EARTH_A/mergedChr/noNA_noMAFfilter.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/PEAC/mergedChr/noNA_noMAFfilter.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/Vienna/mergedChr/noNA_noMAFfilter.assoc.logistic',
 '/exports

In [301]:
#os.chdir("/exports/reum/CKe/Pipeline_GO/")
meta_prepare()

EIRA detected!
EIRA:  29258
RAMS:  451312
Madrid:  3825544
Manchester:  4465515
Tacera:  1743407
EARTH_B:  25603
ACPA:  2592367
SERA:  228830
EARTH_A:  2397388
PEAC:  5104976
Vienna:  1342493
EAC:  232032


In [22]:
merge_temp, dict_count = compareA1(out=os.getcwd()+"/compareA1.txt")
print(dict_count)

cols to keep:  ['SNP', 'NMISS', 'A1_TACERA']
cols to keep:  ['SNP', 'NMISS', 'A1_TACERA', 'A1_EAC']
{1: 8062822, 2: 56993}


In [19]:
mainwk=os.getcwd()
mainwk

'/exports/reum/CKe/Pipeline_GO'

In [12]:
autoCopyPaste(end='_noNA_maf_meta.txt',cwd=mainwk,to=mainwk+"/RunMeta/")

In [15]:
unifyA1(compareFile=mainwk+"/compareA1.txt",metaDir=mainwk+"/RunMeta/")

/tmp/ipykernel_1439190/3227448226.py:1: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  unifyA1(compareFile=mainwk+"/compareA1.txt",metaDir=mainwk+"/RunMeta/")


get dict_final!
work done on cohort  EAC  count:  7541644
work done on cohort  TACERA  count:  4054112


In [287]:
METALresult = pd.read_csv('METALresult_A1corr_maf_withPC1.TBL',sep='\t')

In [294]:
METALresult

,MarkerName,BP,CHR,Effect,StdErr,P-value,Direction
0,11:16274555:C,16274555,11,-0.0743,0.1649,0.65230,??????????-??
1,6:73947097:T,73947097,6,1.4598,1.3680,0.28590,???????+?+???
2,1:62947229:C,62947229,1,-0.1400,0.2525,0.57920,?-+???????+-?
3,6:170451779:T,170451779,6,-1.0692,1.5730,0.49670,??-??????????
4,10:8380278:C,8380278,10,-0.0354,0.2040,0.86220,??????????-??
...,...,...,...,...,...,...,...
9712778,16:62295269:T,62295269,16,-0.0174,0.1279,0.89190,??-????--?+-+
9712779,2:233111433:A,233111433,2,0.0136,0.1296,0.91650,?--?????+?++-
9712780,18:39747300:G,39747300,18,0.1657,0.0979,0.09059,??-+?+??+?+-?
9712781,8:130860372:A,130860372,8,-0.2454,0.4014,0.54100,??????????-??


In [293]:
# METALresult.drop(["Allele1","Allele2"],axis=1,inplace=True)
CHR = METALresult['MarkerName'].str.split(":",expand=True)[0]
BP = METALresult['MarkerName'].str.split(":",expand=True)[1]
METALresult.insert(1,'BP',BP)
METALresult.insert(2,'CHR',CHR)
METALresult.to_csv('METALresult_A1corr_maf_withPC1_noAlleles.TBL',sep='\t',index=0) #remvoe empyt alleles cols so they can be drawn

In [266]:
def filterbySample(comparison,cwd=os.getcwd(),metaJob='METALresult_A1corr_maf_withPC1.TBL',out="METALresult_filtered_13cohorts.TBL"):
    #filter out snps in metal result with less than 4000 patients supporting
    METALresult = pd.read_csv(metaJob,sep='\s+')
    merged = pd.merge(METALresult,comparison,left_on="MarkerName",right_on="SNP",how="left")
    index_postqc = merged[merged['NMISS']>=4000].index
    #index_postqc = merge_temp[merge_temp['NMISS']>=200].index
    CHR = METALresult['MarkerName'].str.split(":",expand=True)[0]
    BP = METALresult['MarkerName'].str.split(":",expand=True)[1]
    METALresult.insert(1,'BP',BP)
    METALresult.insert(2,'CHR',CHR)
    summary_proc = METALresult.iloc[index_postqc,:]
    write_log("saving file to ",cwd+"/"+out)
    summary_proc.to_csv(cwd+"/"+out,sep='\t',index=0)

In [268]:
def ManhandQQplotforMeta(cwd=os.getcwd(),metaJob='METALresult_filtered_13cohorts.TBL'):
    meta_result = cwd+"/"+metaJob
    print("saving plots to: ",cwd)
    os.system("sbatch drawManhMeta.sh" +" "+meta_result+" "+cwd)
    print("Manh&QQ-plot submitted for metal result!")

In [295]:
# mainwk=os.getcwd()
# os.chdir(mainwk+"/RunMeta")
# filterbySample(merge_temp,metaJob="METALresult_A1corr_maf_withPC1.TBL")
ManhandQQplotforMeta(cwd = "/exports/reum/CKe/Pipelinerun_withPC/RunMeta",metaJob="METALresult_A1corr_maf_withPC1_noAlleles.TBL")

saving plots to:  /exports/reum/CKe/Pipelinerun_withPC/RunMeta
Submitted batch job 5740645
Manh&QQ-plot submitted for metal result!


In [277]:
os.getcwd()

'/exports/reum/CKe/Pipelinerun_withPC/RunMeta'

In [10]:
def main(mainwk=os.getcwd(),GWAS=True,metaPrepare=True,makeA1compare=True,correctA1=True,meta=True,postmeta=True,time_to_wait=1200):
    #print("cwd: ",mainwk)
    os.chdir(mainwk)
    if GWAS:
        run_GWAS(cwd=mainwk)
        os.system("sbatch DrawManh.sh")
    if metaPrepare:
        meta_prepare(cwd=mainwk)
        if not os.path.exists(mainwk+"/RunMeta/"):
            os.makedirs(mainwk+"/RunMeta/")
            print("created RunMeta folder!") # BE ADVISED: THIS IS NOT ORIGINALLY USED RunMeta location! check CKE/generic-metal/RunMeta for legacy files.
    if makeA1compare:
        autoCopyPaste(end='_noNA_maf_meta.txt',cwd=mainwk,to=mainwk+"/RunMeta/")
        merge_temp, dict_count = compareA1(cwd=mainwk+"/RunMeta",out=mainwk+"/compareA1.txt")
        print(dict_count)
    if correctA1:    
        unifyA1(compareFile=mainwk+"/compareA1.txt",metaDir=mainwk+"/RunMeta/")
    if postmeta:
        try:
            print(merge_temp.shape)
        except NameError:
            merge_temp = pd.read_csv(mainwk+"/compareA1.txt",sep='\t')
    if meta:
        os.chdir(mainwk+"/RunMeta")
        mainwk=os.getcwd()
        os.system("sbatch run_metal.sh")
        if postmeta:
            time.sleep(time_to_wait) #spare some time (20min) to run metal
    if postmeta:
        os.chdir(mainwk+"/RunMeta")
        filterbySample(merge_temp,metaJob="METALresult_refEIRA_A1corr_maf1.TBL")
        ManhandQQplotforMeta()

In [11]:
def parse_args():
    """
        Parses inputs from the commandline.
        :return: inputs as a Namespace object
    """
    parser = argparse.ArgumentParser(description='Generates pipeline')
    # Arguments
    parser.add_argument('-wk', help='working directory',default=os.getcwd())
    parser.add_argument('--GWAS', help='exclude GWAS running',action="store_false")
    parser.add_argument('--metaPrepare', help='exclude metaPrepare procedure',action="store_false")
    parser.add_argument('--makeA1compare', help='exclude makeA1compare procedure',action="store_false")
    parser.add_argument('--correctA1', help='exclude correctA1 procedure',action="store_false")
    parser.add_argument('--meta', help='exclude meta-analysis',action="store_false")
    parser.add_argument('--postmeta', help='exclude post-meta analysis and visualize procedure',action="store_false")
    parser.add_argument('-t','--timetowait', help='time(sec) to wait after meta-analysis before further steps',default=1200)
    return parser.parse_args()

In [12]:
if __name__ == "__main__":
    args = parse_args()
    mainwk = args.wk
    GWAS = args.GWAS
    metaPrepare = args.metaPrepare
    makeA1compare = args.makeA1compare
    correctA1 = args.correctA1
    meta = args.meta
    postmeta = args.postmeta
    time_to_wait = args.timetowait
    print("Hi! How's your miserable life today? Luckily for this GwasMeta Run, the parameters of your choice: \n")
    print("mainWorkingDir: ",mainwk,"GWAS:",GWAS,"METAPREPARE:",metaPrepare,"MAKEA1COMPARE:",makeA1compare,"CORRECTA1:",correctA1,"META:",meta,"TIME TO WAIT FOR METAL(s):",time_to_wait,"POST META:",postmeta)
    main(mainwk,GWAS,metaPrepare,makeA1compare,correctA1,meta,postmeta,time_to_wait)

usage: ipykernel_launcher.py [-h] [-wk WK] [--GWAS] [--metaPrepare]
                             [--makeA1compare] [--correctA1] [--meta]
                             [--postmeta] [-t TIMETOWAIT]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/cke/.local/share/jupyter/runtime/kernel-92f9e023-4d5e-485a-85bd-826740808375.json


SystemExit: 2

/home/cke/.conda/envs/CKe/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [11]:
try:
    print(merge_temp.shape)
except NameError:
    merge_temp = pd.read_csv(mainwk+"/compareA1.txt",sep='\t')

/home/cke/.conda/envs/CKe/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
os.chdir(mainwk+"/RunMeta")
filterbySample(merge_temp,metaJob="METALresult_refEIRA_A1corr_maf1.TBL")
ManhandQQplotforMeta()

Manh&QQ-plot submitted for metal result!


sbatch: error: Unable to open file drawManhMeta.sh
